## Wave conditions examples

input file: long term time series

### 3.6.3 Joint ditribution of significant wave height and period



In [ ]:
import pandas as pd 
import numpy as np  
import matplotlib.pylab as plt 
import scipy  
import scipy.misc
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 8, 6
import datetime
import scipy.stats as sts
plt.xkcd()

In [ ]:
MetoData = pd.read_pickle('H1.pkl')

In [ ]:
U10 = MetoData['H1: Wind speed']
D10 = MetoData['H1: Wind direction']

Hm0 = MetoData['H1: Sign. Wave Height']
MWD = MetoData['H1: Mean Wave Direction']
Tp  = MetoData['H1: Peak Wave Period']
T01 = MetoData['H1: Wave Period, T01']
T02 = MetoData['H1: Wave Period, T02']

WL = MetoData['H1: Surface elevation']
U  = MetoData['H1: Current velocity, U']
V  = MetoData['H1: Current velocity, V']


MetoData.head()

In [ ]:
# Plot wind rose and wave rose
from windrose import WindroseAxes

ax = WindroseAxes.from_ax()
ax.bar(D10, U10, normed=True, opening=0.8, edgecolor='white')
ax.set_legend()

In [ ]:
# Plot  wave rose


ax = WindroseAxes.from_ax()
ax.bar(MWD, Hm0, normed=True, opening=0.8, edgecolor='white')
ax.set_legend()

In [ ]:
# annual maximum
amax = MetoData.loc[MetoData.groupby(MetoData.index.year)['H1: Sign. Wave Height'].idxmax()]
Hm0_amax = amax['H1: Sign. Wave Height']


plt.plot(MetoData['H1: Sign. Wave Height'], label = 'Hm0')
plt.plot(amax['H1: Sign. Wave Height'],'r*', label = 'Hm0 Annual maximum')
plt.legend()

In [ ]:
# pip install reliability

from reliability.Fitters import Fit_Weibull_3P

best= Fit_Weibull_3P(failures=Hm0_amax.values,show_probability_plot =True)
#plt.xscale('linear')
#plt.xlim([0,10])
HAlpha=best.alpha
HBeta=best.beta
HGamma=best.gamma
#plt.xscale('linear')
#plt.yscale('linear')
best
del best

In [ ]:
from reliability.Fitters import Fit_Lognormal_2P
LongtermTz=MetoceanData[Tpname].values
T1_2=MetoceanData[(MetoceanData[Hsname] < 2.7 )& (MetoceanData[Hsname] > 2.5)]

best= Fit_Lognormal_2P(failures=T1_2[Tpname].values,print_results=True,show_probability_plot =True)
plt.xscale('linear')
plt.yscale('linear')


In [ ]:
# this block calculate mu and sigma depend on Hs
LongtermTp=MetoceanData[Tpname].values

HsMax=max(Hs)
TzMax=max(Tp)

H_s=np.linspace(0,HsMax*0.8,num=51)
H_s[-1]=10
T_p=np.linspace(0,TzMax*0.9,num=61)
Condi_mu=np.zeros(len(H_s))
Condi_sigma=np.zeros(len(H_s))
for ih in range(1,len(H_s)-1):
    down_lim=H_s[ih]
    up_lim=H_s[ih+1]
    T1_2=MetoceanData[(MetoceanData[Hsname] < up_lim )& (MetoceanData[Hsname] >= down_lim)]
    best= Fit_Lognormal_2P(failures=T1_2[Tpname].values,print_results=False,show_probability_plot =False)
    Condi_mu[ih]=best.mu
    Condi_sigma[ih]=best.sigma;
Condi_mu[-1]=Condi_mu[-2]
Condi_mu[0]=Condi_mu[1]
Condi_sigma[-1]=Condi_sigma[-2]
Condi_sigma[0]=Condi_sigma[1]


In [ ]:
#fit mu to get a0,a1,a2
afit,out=scipy.optimize.curve_fit(lambda t,a0,a1,a2: a0+a1*t**a2,  H_s,  Condi_mu)
#bfit=scipy.optimize.curve_fit(lambda t,b0,b1,b2: -(b0+b1*np.exp(b2*t)),  H_s,  Condi_sigma)
plt.plot(H_s,Condi_mu,'r*', label = 'Data')
plt.plot(H_s,afit[0]+afit[1]*H_s**afit[2], label = 'Fit')
afit
plt.legend()

In [ ]:
#fit mu to get a0,a1,a2
#P0 in curve fitting is important 
bfit,_=scipy.optimize.curve_fit(lambda t,b0,b1,b2:(b0+b1*np.exp(b2*t)),  H_s,  Condi_sigma, p0=(0.07, 0.1288,-0.0214),maxfev=3000)
plt.plot(H_s,Condi_sigma,'r*', label = 'Data')
plt.plot(H_s,bfit[0]+bfit[1]*np.exp(bfit[2]*H_s), label = 'Fit')
bfit
plt.legend()

In [ ]:
#
a0=afit[0]
a1=afit[1]
a2=afit[2]
b0=bfit[0]
b1=bfit[1]
b2=bfit[2]

H_s=np.linspace(0,10,num=101)
T_p=np.linspace(0,15,num=151)
HT_dens=np.zeros([len(H_s),len(T_p)])


T_condi_mu=a0+a1*H_s**a2
T_condi_sigma=b0+b1*np.exp(1)**(b2*H_s)
fHs=HBeta/HAlpha*((H_s-HGamma)/HAlpha)**(HBeta-1)*np.exp(-((H_s-HGamma)/HAlpha)**HBeta)
#HT_dens
#H_s[1]
#plt.plot(H_s,fHs)
#plt.hist(LongtermHs,density=True,bins=50)
#fHs=HBeta/HAlpha*((H_s[ih]-HGamma)/HAlpha)**(HBeta-1)*np.exp(-((H_s[ih]-HGamma)/HAlpha)**HBeta);

for ih in range(1,len(H_s)):
    for it in range(1,len(T_p)):
        T_condi=1./(T_condi_sigma[ih]*T_p[it]*np.sqrt(2*np.pi))*np.exp(-((np.log(T_p[it])-T_condi_mu[ih])**2)/((2*T_condi_sigma[ih])**2))
        HT_dens[ih,it]=T_condi*fHs[ih];
        


In [ ]:
import matplotlib.pyplot as plt
X, Y = np.meshgrid(T_p,H_s)

plt.scatter(Tp,Hs, cmap ='Greys' , alpha=0.2)
cs=plt.contourf(X, Y,HT_dens,levels=[ 0.1, 0.2,0.3,0.4,.5,.6,.7,.8,.9,],cmap ='coolwarm',alpha=0.3)
plt.colorbar()
plt.xlim(0,25)
plt.ylim(0,10)

plt.xlabel('Wave Period Tp [s]')
plt.ylabel('Wave Height Hs [m]')

plt.clabel(cs)




In [ ]:
from viroconcom.fitting import Fit
from viroconcom.contours import IFormContour
#### Environmental Countour
plt.scatter(Hs, Tp)
plt.xlabel('significant wave height [m]')
plt.ylabel('spectral peak period [s]')
plt.show()

In [ ]:
dist_description_0 = {'name': 'Weibull_3p', 'dependency': (None, None, None), 'width_of_intervals': 1}
dist_description_1 = {'name': 'Lognormal_SigmaMu', 'dependency': (0, None, 0), 'functions': ('exp3', None, 'power3')}
#distribution depends on the variable

In [ ]:
my_fit = Fit((Hs, Tp), (dist_description_0, dist_description_1))


In [ ]:
# For panel A: use a histogram.
fig = plt.figure(figsize=(9, 4.5))
ax_1 = fig.add_subplot(121)
param_grid = my_fit.multiple_fit_inspection_data[0].scale_at
plt.hist(my_fit.multiple_fit_inspection_data[0].scale_samples[0], density=1,
         label='sample')
shape = my_fit.mul_var_dist.distributions[0].shape(0)
scale = my_fit.mul_var_dist.distributions[0].scale(0)
plt.plot(np.linspace(0, 20, 100),
         sts.weibull_min.pdf(np.linspace(0, 20, 100), c=shape, loc=0,
                             scale=scale),
         label='fitted Weibull distribution')
plt.xlabel('significant wave height [m]')
plt.ylabel('probability density [-]')
plt.legend()
# For panel B: use a Q-Q plot.
ax_2 = fig.add_subplot(122)
sts.probplot(my_fit.multiple_fit_inspection_data[0].scale_samples[0],
             sparams=(shape, 0, scale), dist=sts.weibull_min, plot=plt)
ax_2.get_lines()[0].set_markerfacecolor('#1f77ba') # Adapt to v2.0 colors
ax_2.get_lines()[0].set_markeredgecolor('#1f77ba') # Adapt to v2.0 colors
ax_2.get_lines()[1].set_color('#ff7f02') # Adapt to v2.0 colors
plt.title("")
plt.xlabel('theoretical quantiles [m]')
plt.ylabel('data quantiles [m]')
plt.show()

In [ ]:
fig = plt.figure(figsize=(10, 8))
ax_1 = fig.add_subplot(221)
title1 = ax_1.set_title('Tp-Distribution for 0≤Hs<2')
param_grid = my_fit.multiple_fit_inspection_data[1].scale_at
ax1_hist = ax_1.hist(my_fit.multiple_fit_inspection_data[1].scale_samples[0], density=1)
shape = my_fit.mul_var_dist.distributions[1].shape(0)
scale = my_fit.mul_var_dist.distributions[1].scale(param_grid[0])
ax1_plot = ax_1.plot(np.linspace(0, 20, 100), sts.lognorm.pdf(np.linspace(0, 20, 100), s=shape, scale=scale))

ax_2 = fig.add_subplot(222)
title2 = ax_2.set_title('Tp-Distribution for 2≤Hs<4')
ax2_hist = ax_2.hist(my_fit.multiple_fit_inspection_data[1].scale_samples[1], density=1)
shape = my_fit.mul_var_dist.distributions[1].shape(0)
scale = my_fit.mul_var_dist.distributions[1].scale(param_grid[1])
ax2_plot = ax_2.plot(np.linspace(0, 20, 100), sts.lognorm.pdf(np.linspace(0, 20, 100), s=shape, scale=scale))

ax_3 = fig.add_subplot(223)
title3 = ax_3.set_title('Tp-Distribution for 4≤Hs<6')
ax3_hist = ax_3.hist(my_fit.multiple_fit_inspection_data[1].scale_samples[2], density=1)
shape = my_fit.mul_var_dist.distributions[1].shape(0)
scale = my_fit.mul_var_dist.distributions[1].scale(param_grid[2])
ax3_plot = ax_3.plot(np.linspace(0, 20, 100), sts.lognorm.pdf(np.linspace(0, 20, 100), s=shape, scale=scale))
ax_3.set_xlabel('spectral peak period [s]')

ax_4 = fig.add_subplot(224)
title4 = ax_4.set_title('Tp-Distribution for 6≤Hs<8')
ax4_hist = ax_4.hist(my_fit.multiple_fit_inspection_data[1].scale_samples[3], density=1)
shape = my_fit.mul_var_dist.distributions[1].shape(0)
scale = my_fit.mul_var_dist.distributions[1].scale(param_grid[3])
ax4_plot = ax_4.plot(np.linspace(0, 20, 100), sts.lognorm.pdf(np.linspace(0, 20, 100), s=shape, scale=scale))
ax_4.set_xlabel('spectral peak period [s]')
plt.show()

fig = plt.figure()
x_1 = np.linspace(0, 12, 100)
plt.plot(param_grid, my_fit.multiple_fit_inspection_data[1].scale_value, 'x',
         label='discrete scale values')
plt.plot(x_1, my_fit.mul_var_dist.distributions[1].scale(x_1),
         label='fitted dependence function')
plt.xlabel('significant wave height [m]')
plt.ylabel('scale parameter (Tp-distribution)')
plt.legend()
plt.show()

In [ ]:

fig = plt.figure()
x_1 = np.linspace(0, 12, 100)
plt.plot(param_grid, my_fit.multiple_fit_inspection_data[1].scale_value, 'x',
         label='discrete scale values')
plt.plot(x_1, my_fit.mul_var_dist.distributions[1].scale(x_1),
         label='fitted dependence function')
plt.xlabel('significant wave height [m]')
plt.ylabel('scale parameter (Tp-distribution) [-]')
plt.legend()
plt.show()

In [ ]:
iform_contour = IFormContour(my_fit.mul_var_dist, 10, 1, 1000)
plt.scatter(Hs, Tp, label='sample')
plt.plot(iform_contour.coordinates[0][0], iform_contour.coordinates[0][1],
            ':k', label='10year')
iform_contour = IFormContour(my_fit.mul_var_dist, 50, 1, 1000)
plt.plot(iform_contour.coordinates[0][0], iform_contour.coordinates[0][1],
            ':b', label='50year')
iform_contour = IFormContour(my_fit.mul_var_dist, 100, 1, 1000)
plt.plot(iform_contour.coordinates[0][0], iform_contour.coordinates[0][1],
            ':r', label='100year')

plt.xlabel('significant wave height [m]',fontsize= 20)
plt.ylabel('spectral peak period [s]',fontsize= 20)
plt.legend()
plt.show()

In [ ]:
iform_contour = IFormContour(my_fit.mul_var_dist, 10, 1, 50)
plt.scatter(Hs, Tp, label='sample')
plt.plot(iform_contour.coordinates[0][0], iform_contour.coordinates[0][1],
            ':k', label='10year')
iform_contour = IFormContour(my_fit.mul_var_dist, 50, 1, 50)
plt.plot(iform_contour.coordinates[0][0], iform_contour.coordinates[0][1],
            ':b', label='50year')
iform_contour = IFormContour(my_fit.mul_var_dist, 100, 1, 50)
plt.plot(iform_contour.coordinates[0][0], iform_contour.coordinates[0][1],
            ':r', label='100year')

plt.xlabel('significant wave height [m]',fontsize= 20)
plt.ylabel('spectral peak period [s]',fontsize= 20)
plt.legend()
plt.show()